# Universidade NOVA de Lisboa
## Demo 2: NETCONF e YANG com ```ncclient``` (Python 🐍)

Ferramentas de conexão via SSH são úteis para automatizar operações em diferentes tipos de dispositivos. No enquanto, a simulação de operações via CLI tem muitos problemas e riscos:

- 📦 Transações. Tudo o nada! O que acontece quando uma parte da nova configuração está errada e precisamos de fazer rollback?
- 👎 Gestão de erros
- 📄 Modelo de dados na mesma plataforma e vendor
- ✍🏼 Sintaxe dos comandos muda entre versões

A solução é o uso consistente de protocolos de programabilidade e modelos de dados YANG para a operação dos nossos dispositivos

### O que vamos fazer nesta demo

- Explorar a biblioteca de Python ```ncclient``` para a interação via NETCONF com um dispositivo Cisco
- Explorar as capabilidades do dispositivo
- Explorar a estrutura de um modelo de dados
- Ler e modificar configurações do dispositivo

Primeiro, vamos importar as seguintes bibliotecas

In [2]:
from ncclient import manager
import re

Neste laboratorio temos um dispositivo Cisco IOSXR. O paso seguinte é a especificação dos parâmetros para a conexão

In [3]:
CISCO_IP = "192.168.10.4"
CISCO_PORT = 22
CISCO_USERNAME = "vagrant"
CISCO_PASSWORD = "vagrant"

Agora, vamos fazer a conexão via SSH com o dispositivo

In [5]:
xr = manager.connect(
    host = CISCO_IP,
    port = CISCO_PORT,
    username = CISCO_USERNAME,
    password = CISCO_PASSWORD,
	allow_agent=False,
	look_for_keys=False,
	hostkey_verify=False,
	unknown_host_cb=True)

Estamos prontos para interactuar com o nosso dispositivo via NETCONF.

O primeiro cenário é o seguinte - O qué acontece se o troco de versão de firmware no dispositivo mudou algumas das funcionalidades? Pode ser que algum comando CLI tenha mudado ...

Para validar qualquer mudançã relevante para a nossa gestão, é possível consultar as capacidades do dispositivo e olhar se temos alguma versão nova nos modelos de dados. Caso de ter-la, é possível saber qual foi a parte que mudou com un ```diff``` entre versões.


In [6]:
for c in xr.server_capabilities:
       print c

http://cisco.com/ns/yang/Cisco-IOS-XR-l2-eth-infra-cfg?module=Cisco-IOS-XR-l2-eth-infra-cfg&revision=2015-11-09
http://openconfig.net/yang/vlan?module=openconfig-vlan&revision=2015-10-09&deviation=cisco-xr-openconfig-vlan-deviations
http://cisco.com/ns/yang/Cisco-IOS-XR-pmengine-cfg?module=Cisco-IOS-XR-pmengine-cfg&revision=2015-11-09
http://cisco.com/ns/yang/Cisco-IOS-XR-mpls-ldp-oper?module=Cisco-IOS-XR-mpls-ldp-oper&revision=2015-11-09
http://cisco.com/ns/yang/Cisco-IOS-XR-clns-isis-oper?module=Cisco-IOS-XR-clns-isis-oper&revision=2015-11-09
http://cisco.com/ns/yang/cisco-xr-openconfig-interfaces-deviations?module=cisco-xr-openconfig-interfaces-deviations&revision=2016-05-16
http://cisco.com/ns/yang/Cisco-IOS-XR-ip-iarm-v6-oper?module=Cisco-IOS-XR-ip-iarm-v6-oper&revision=2015-11-09
http://cisco.com/ns/yang/Cisco-IOS-XR-plat-chas-invmgr-oper?module=Cisco-IOS-XR-plat-chas-invmgr-oper&revision=2015-01-07
http://cisco.com/ns/yang/Cisco-IOS-XR-ipv6-acl-cfg?module=Cisco-IOS-XR-ipv6-acl-c

São muitos, sim! Mas é possível filtrar a lista com regex. Neste exemplo, estamos a procurar o modelo de dados que tem a ver com a configuração de rotas estáticas.

In [7]:
for c in xr.server_capabilities:
       model = re.search('module=([^&]*static[^&]*)&', c)
       if model is not None:
           print model.group(1)

Cisco-IOS-XR-mpls-static-cfg
Cisco-IOS-XR-ip-static-cfg
Cisco-IOS-XR-mpls-static-oper


Eureka! O modelo que eu quero consultar é ```Cisco-IOS-XR-ip-static-cfg```

In [14]:
data_model_payload = xr.get_schema("Cisco-IOS-XR-ip-static-cfg")
print data_model_payload.data

module Cisco-IOS-XR-ip-static-cfg {

  /*** NAMESPACE / PREFIX DEFINITION ***/

  namespace "http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg";


  prefix "ip-static-cfg";

  /*** LINKAGE (IMPORTS / INCLUDES) ***/

  import ietf-inet-types { prefix "inet"; }

  import Cisco-IOS-XR-types { prefix "xr"; }

  /*** META INFORMATION ***/

  organization "Cisco Systems, Inc.";

  contact
    "Cisco Systems, Inc.
     Customer Service

     Postal: 170 West Tasman Drive
     San Jose, CA 95134

     Tel: +1 800 553-NETS

     E-mail: cs-yang@cisco.com";

  description 
    "This module contains a collection of YANG definitions
     for Cisco IOS-XR ip-static package configuration.

     This module contains definitions
     for the following management objects:
       router-static: This class represents router static
         configuration

     Copyright (c) 2013-2016 by Cisco Systems, Inc.
     All rights reserved.";

  revision "2015-09-10" {
    description
      "Descriptions updated

Ás vezes não é fácil ler um modelo de dados com a sintaxe em YANG, especialmente quando o modelo é muito grande e tem muitas estruturas. A ferramenta ```pyang``` permite a visualização da estructura de uma maneira gráfica.

Neste exemplo, vamos guardar o nosso modelo de dados num arquivo e depóis vamos gerar o gráfico para melhor visualização.

In [17]:
with open("Cisco-IOS-XR-ip-static-cfg.yang", "w") as f:
    f.write(data_model_payload.data)
    
!pyang -f tree --tree-depth 5 Cisco-IOS-XR-ip-static-cfg.yang

Cisco-IOS-XR-ip-static-cfg.yang:14: error: module "Cisco-IOS-XR-types" not found in search path
module: Cisco-IOS-XR-ip-static-cfg
    +--rw router-static
       +--rw vrfs
       |  +--rw vrf* [vrf-name]
       |     +--rw vrf-name          xr:Cisco-ios-xr-string
       |     +--rw address-family
       |        +--rw vrfipv4
       |        |     ...
       |        +--rw vrfipv6
       |              ...
       +--rw default-vrf
       |  +--rw address-family
       |     +--rw vrfipv4
       |     |  +--rw vrf-unicast
       |     |  |     ...
       |     |  +--rw vrf-multicast
       |     |        ...
       |     +--rw vrfipv6
       |        +--rw vrf-unicast
       |        |     ...
       |        +--rw vrf-multicast
       |              ...
       +--rw maximum-routes
          +--rw ipv6-routes?   uint32
          +--rw ipv4-routes?   uint32


O qué aconteceu? Este modelo de dados tem uma dependência com outro modelo.

Vamos descarregar o modelo ```Cisco-IOS-XR-types.yang``` e tentar de novo.

In [21]:
with open("Cisco-IOS-XR-types.yang", "w") as f:
    f.write(xr.get_schema("Cisco-IOS-XR-types").data)
    
!pyang -f tree --tree-depth 10 Cisco-IOS-XR-ip-static-cfg.yang

module: Cisco-IOS-XR-ip-static-cfg
    +--rw router-static
       +--rw vrfs
       |  +--rw vrf* [vrf-name]
       |     +--rw vrf-name          xr:Cisco-ios-xr-string
       |     +--rw address-family
       |        +--rw vrfipv4
       |        |  +--rw vrf-unicast
       |        |  |  +--rw topologies
       |        |  |  |  +--rw topology* [topology-name]
       |        |  |  |     +--rw topology-name            xr:Cisco-ios-xr-string
       |        |  |  |     +--rw vrf-prefix-topologies
       |        |  |  |        +--rw vrf-prefix-topology* [prefix prefix-length]
       |        |  |  |              ...
       |        |  |  +--rw vrf-prefixes
       |        |  |  |  +--rw vrf-prefix* [prefix prefix-length]
       |        |  |  |     +--rw prefix                inet:ip-address-no-zone
       |        |  |  |     +--rw prefix-length         uint32
       |        |  |  |     +--rw vrf-route
       |        |  |  |     |  +--rw vrf-next-hop-table
       |        |  |  | 

       |     |  |  |           +--rw vrf-next-hop-next-hop-address-explicit-path-name* [next-hop-address explicit-path-name]
       |     |  |  |           |     ...
       |     |  |  |           +--rw vrf-next-hop-explicit-path-name* [explicit-path-name]
       |     |  |  |                 ...
       |     |  |  +--rw default-topology
       |     |  |     +--rw vrf-prefix-topologies
       |     |  |        +--rw vrf-prefix-topology* [prefix prefix-length]
       |     |  |           +--rw prefix                inet:ip-address-no-zone
       |     |  |           +--rw prefix-length         uint32
       |     |  |           +--rw vrf-route
       |     |  |           |  +--rw vrf-next-hop-table
       |     |  |           |        ...
       |     |  |           +--rw vrf-recurse-routes
       |     |  |           |  +--rw vrf-recurse-route* [vrf-name]
       |     |  |           |        ...
       |     |  |           +--rw vrf-seg-route
       |     |  |              +--rw segme

No exemplo, é possível trocar o valor de ```--tree-depth``` para gerar um gráfico com maior profundidade nas estruturas.

Pronto! Agora sabemos qual é a estrutura para a leitura e configuração das rotas estáticas neste dispositivo.

Vamos ler agora as configurações neste dispositivo. NETCONF tem o controlo de três tipos diferentes:

- running-config
- startup-config
- candidate-config

Neste exemplo, vamos ler a configuração tipo ```running-config```

In [19]:
print xr.get_config(source='running')

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:0ecd631a-434a-430f-b45f-385472aa9a17" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <ip xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-tcp-cfg">
   <cinetd>
    <services>
     <vrfs>
      <vrf>
       <vrf-name>default</vrf-name>
       <ipv4>
        <telnet>
         <tcp>
          <maximum-server>10</maximum-server>
         </tcp>
        </telnet>
       </ipv4>
      </vrf>
     </vrfs>
    </services>
   </cinetd>
  </ip>
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>
    <active>act</active>
    <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
    <ipv4-network xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-io-cfg">
     <addresses>
      <dhcp></dhcp>
     </addresses>
    </ipv4-network>
   </interface-configuration>
   <interface-configuration>
    <active>act</active>
    <interfac

Com o modelo de dados do arquivo ```Cisco-IOS-XR-ip-static-cfg.yang```, nós sabemos cómo filtrar a configuração geral, de tal maneira que o dispositivo entregue somente as rotas estáticas.

In [20]:
filter = '''<router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">'''
routes=xr.get_config(source='running', filter=('subtree', filter))
print routes

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:e2e0401e-65f2-4da2-bf29-0710487cc7bb" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
   <default-vrf>
    <address-family>
     <vrfipv4>
      <vrf-unicast>
       <vrf-prefixes>
        <vrf-prefix>
         <prefix>0.0.0.0</prefix>
         <prefix-length>0</prefix-length>
         <vrf-route>
          <vrf-next-hop-table>
           <vrf-next-hop-interface-name-next-hop-address>
            <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
            <next-hop-address>10.0.2.2</next-hop-address>
           </vrf-next-hop-interface-name-next-hop-address>
          </vrf-next-hop-table>
         </vrf-route>
        </vrf-prefix>
        <vrf-prefix>
         <prefix>1.1.1.0</prefix>
         <prefix-length>24</prefix-length>
         <vrf-route>
          <vrf-next-hop-table>
           <

Este payload pode ser o modelo para inserir novas configurações na coleção das rotas estáticas do dispositivo.

Mas vamos fazer uso dos diferentes tipos de configurações que NETCONF pode operar. Lembras-te deles? (```running-config, startup-config, candidate-config```). O dispositivo tem rcps internos que permitem a validação da nova configuração antes de move-la ao ```running-config```.

Primeiro, vamos crear o novo modelo e modificar a configuração de ```candidate-config```

In [23]:
edit_data = '''
<config>
<router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
  <default-vrf>
    <address-family>
    <vrfipv4>
    <vrf-unicast>
    <vrf-prefixes>
        <vrf-prefix>
        <prefix>2.2.2.0</prefix>
        <prefix-length>25</prefix-length>
        <vrf-route>
        <vrf-next-hop-table>
        <vrf-next-hop-next-hop-address>
            <next-hop-address>192.168.10.5</next-hop-address>
        </vrf-next-hop-next-hop-address>
        </vrf-next-hop-table>
        </vrf-route>
        </vrf-prefix>
    </vrf-prefixes>
    </vrf-unicast>
    </vrfipv4>
    </address-family>
  </default-vrf>
</router-static>
</config>
'''

test=xr.edit_config(edit_data, target='candidate', format='xml')
print test

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:226882bd-b559-45ca-9a6d-c8a59eade571" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



Se a configuração é válida, vamos mover para o ```running-config``` com o rpc ```commit```

In [25]:
if test.ok: 
    print xr.commit()
    
routes=xr.get_config(source='running', filter=('subtree', filter))
print routes

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:5d4ee31b-9156-4df2-b75b-883326b2bef7" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:a97c5153-1739-4e90-bf7d-8a254fdacd99" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
   <default-vrf>
    <address-family>
     <vrfipv4>
      <vrf-unicast>
       <vrf-prefixes>
        <vrf-prefix>
         <prefix>0.0.0.0</prefix>
         <prefix-length>0</prefix-length>
         <vrf-route>
          <vrf-next-hop-table>
           <vrf-next-hop-interface-name-next-hop-address>
            <interface-name>MgmtEth0/RP0/CPU0/0</interface-name>
            <next-hop-address>10.0.2.2</next-hop-address>
           </vrf-next-hop-interface-name-next-hop-address>
          </vrf-next-hop-

Finalmente, vamos eliminar a rota estática e terminar a sessão NETCONF

In [ ]:
edit_data = '''
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0">
<router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
  <default-vrf>
    <address-family>
    <vrfipv4>
    <vrf-unicast>
    <vrf-prefixes>
        <vrf-prefix xc:operation="delete">
        <prefix>2.2.2.0</prefix>
        <prefix-length>25</prefix-length>
        </vrf-prefix>
    </vrf-prefixes>
    </vrf-unicast>
    </vrfipv4>
    </address-family>
  </default-vrf>
</router-static>
</config>
'''

print xr.edit_config(edit_data, target='candidate', format='xml')
print xr.commit()
xr.close_session()